In [1]:
# CAMERA CALIBRATION

# IMPORTS
#---------------------------

import cv2
import glob
import numpy

#---------------------------

# VARIABLES
#---------------------------

# paths
mainPath = os.getcwd()
imagesPath = mainPath + '\Images\*.jpg'

# chessboard dimension
chessboard = (9, 6)

# images
fileImages = glob.glob(imagesPath)

# criteria termination
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 28, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(8,6,0)
objectPoint = numpy.zeros((chessboard[0] * chessboard[1], 3), numpy.float32)
objectPoint[:,:2] = numpy.mgrid[0:chessboard[0], 0:chessboard[1]].T.reshape(-1, 2)

# arrays to store object points and image points from all the images
objectPoints = [] # 3D points in real world space
imagePoints = [] # 2D points in image plane

#---------------------------

# CODE
#---------------------------

for fileImage in fileImages:
    image = cv2.imread(fileImage)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (chessboard[0], chessboard[1]), None)
    
    # if found, add object points, image points (after refining them)
    if ret == True:
        objectPoints.append(objectPoint)
        corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        imagePoints.append(corners)
        
        # draw and display the corners
        cv2.drawChessboardCorners(image, (chessboard[0], chessboard[1]), corners2, ret)
        cv2.imshow('IMG', image)
        cv2.waitKey(500)

cv2.destroyAllWindows()

# calibration
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objectPoints, imagePoints, gray.shape[::-1], None, None)

# print results
print('RET:')
print(ret)
print()
print('MTX:')
print(mtx)
print()
print('DIST:')
print(dist)
print()
print('RVECS:')
print(rvecs)
print()
print('TVECS:')
print(tvecs)
print()

# save camera calibration to files
numpy.save(mainPath + '\mtx.npy', mtx)
numpy.save(mainPath + '\dist.npy', dist)

#---------------------------

RET:
0.769688779106233

MTX:
[[951.50264721   0.         645.78565699]
 [  0.         950.60546197 376.17484308]
 [  0.           0.           1.        ]]

DIST:
[[-5.11023201e-02  1.07492444e+00 -3.68309691e-04  2.03247023e-03
  -3.31324985e+00]]

RVECS:
(array([[ 0.13796847],
       [-0.03039333],
       [ 3.12974848]]), array([[-1.05126499],
       [ 0.10349084],
       [ 2.87448431]]), array([[-1.01906168],
       [ 0.09087752],
       [-2.95284127]]), array([[ 0.13478616],
       [ 1.17039963],
       [-2.88671248]]), array([[ 0.02645107],
       [-0.93104259],
       [-2.98589551]]), array([[ 0.63631331],
       [-1.17027678],
       [ 2.46275242]]), array([[-0.67940008],
       [-0.84345659],
       [ 2.82992481]]), array([[-0.22912747],
       [ 0.38535449],
       [ 2.7238135 ]]), array([[-0.80097383],
       [-0.01849673],
       [-2.7142527 ]]), array([[0.14109123],
       [0.08460765],
       [2.41509732]]), array([[0.08022359],
       [0.10482372],
       [1.57370352]]), 

In [2]:
# IMAGE 3D RECONSTRUCTION

# IMPORTS
#---------------------------

from datetime import datetime
import cv2
import glob
import numpy
import os

#---------------------------

# FUNCTIONS
#---------------------------

def draw(img, corners, imgpts):
    corner = tuple(corners[0].ravel())
    img = cv2.line(img, (int(corner[0]), int(corner[1])), (int(tuple(imgpts[0].ravel())[0]),int(tuple(imgpts[0].ravel())[1])), (255,0,0), 5)
    img = cv2.line(img, (int(corner[0]), int(corner[1])), (int(tuple(imgpts[1].ravel())[0]),int(tuple(imgpts[1].ravel())[1])), (0,255,0), 5)
    img = cv2.line(img, (int(corner[0]), int(corner[1])), (int(tuple(imgpts[2].ravel())[0]),int(tuple(imgpts[2].ravel())[1])), (0,0,255), 5)
    return img

def draw2(img, imgpts):
    imgpts = numpy.int32(imgpts).reshape(-1, 2)

    # draw ground floor in green
    img = cv2.drawContours(img, [imgpts[:4]], -1, (0, 255, 0), -3)

    # draw pillars in blue color
    for i,j in zip(range(4), range(4, 8)):
        img = cv2.line(img, tuple(imgpts[i]), tuple(imgpts[j]), (255), 3)

    # draw top layer in red color
    img = cv2.drawContours(img, [imgpts[4:]], -1, (0, 0, 255), 3)

    return img

#---------------------------

# VARIABLES
#---------------------------

# paths
mainPath = os.getcwd()
imagesInputPath = mainPath + '\Images\*.jpg'
imagesOutputPath = mainPath + '\Images_3D'

# chessboard dimension
chessboard = (9, 6)

# images
fileImages = glob.glob(imagesInputPath)

# criteria termination
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 28, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(5,8,0)
objectPoint = numpy.zeros((chessboard[0] * chessboard[1], 3), numpy.float32)
objectPoint[:,:2] = numpy.mgrid[0:chessboard[0], 0:chessboard[1]].T.reshape(-1, 2)

#axis = numpy.float32([[3,0,0], [0,3,0], [0,0,-3]]).reshape(-1,3)
axis = numpy.float32([[0,0,0], [0,3,0], [3,3,0], [3,0,0],
                   [0,0,-3],[0,3,-3],[3,3,-3],[3,0,-3]])

# load camera calibration from files
mtx = numpy.load(mainPath + '\mtx.npy')
dist = numpy.load(mainPath + '\dist.npy')

# index
ind = 0
#---------------------------

# CODE
#---------------------------

# print results
print('MTX:')
print(mtx)
print()
print('DIST:')
print(dist)
print()

for fileImage in fileImages:
    ind = ind + 1

    image = cv2.imread(fileImage)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, (chessboard[0], chessboard[1]), None)

    if ret == True:
        corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)

        # Find the rotation and translation vectors.
        _, rvecs, tvecs, inliers = cv2.solvePnPRansac(objectPoint, corners2, mtx, dist)

        # project 3D points to image plane
        imgpts, jac = cv2.projectPoints(axis, rvecs, tvecs, mtx, dist)

        image = draw2(image, imgpts)
        cv2.imshow('IMG', image)
        k = cv2.waitKey(0)

        if k == ord('s') or k == ord('S'):
            now = datetime.now()
            currentTime = now.strftime("%H-%M-%S")
            cv2.imwrite(imagesOutputPath + '\image_3D_' + str(ind) + '_' + currentTime + os.path.splitext(fileImage)[-1], image)

cv2.destroyAllWindows()
#---------------------------

MTX:
[[951.50264721   0.         645.78565699]
 [  0.         950.60546197 376.17484308]
 [  0.           0.           1.        ]]

DIST:
[[-5.11023201e-02  1.07492444e+00 -3.68309691e-04  2.03247023e-03
  -3.31324985e+00]]



In [15]:
# CAMERA 3D RECONSTRUCTION

# IMPORTS
#---------------------------

from datetime import datetime
import cv2
import glob
import numpy
import os

#---------------------------q

# FUNCTIONS
#---------------------------

def draw(img, corners, imgpts):
    corner = tuple(corners[0].ravel())
    img = cv2.line(img, (int(corner[0]), int(corner[1])), (int(tuple(imgpts[0].ravel())[0]),int(tuple(imgpts[0].ravel())[1])), (255,0,0), 5)
    img = cv2.line(img, (int(corner[0]), int(corner[1])), (int(tuple(imgpts[1].ravel())[0]),int(tuple(imgpts[1].ravel())[1])), (0,255,0), 5)
    img = cv2.line(img, (int(corner[0]), int(corner[1])), (int(tuple(imgpts[2].ravel())[0]),int(tuple(imgpts[2].ravel())[1])), (0,0,255), 5)
    return img

def draw2(img, imgpts):
    imgpts = numpy.int32(imgpts).reshape(-1, 2)

    # draw ground floor in green
    img = cv2.drawContours(img, [imgpts[:4]], -1, (0, 255, 0), -3)

    # draw pillars in blue color
    for i,j in zip(range(4), range(4, 8)):
        img = cv2.line(img, tuple(imgpts[i]), tuple(imgpts[j]), (255), 3)

    # draw top layer in red color
    img = cv2.drawContours(img, [imgpts[4:]], -1, (0, 0, 255), 3)

    return img

def rescale_frame(frame, percent=100):
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)

#---------------------------

# VARIABLES
#---------------------------

# paths
mainPath = os.getcwd()
imagesOutputPath = mainPath + '\Camera_3D'

# chessboard dimension
chessboard = (9, 6)

# criteria termination
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 28, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(5,8,0)
objectPoint = numpy.zeros((chessboard[0] * chessboard[1], 3), numpy.float32)
objectPoint[:,:2] = numpy.mgrid[0:chessboard[0], 0:chessboard[1]].T.reshape(-1, 2)

#axis = numpy.float32([[3,0,0], [0,3,0], [0,0,-3]]).reshape(-1,3)
axis = numpy.float32([[0,0,0], [0,3,0], [3,3,0], [3,0,0],
                   [0,0,-3],[0,3,-3],[3,3,-3],[3,0,-3]])

# load camera calibration from files
mtx = numpy.load(mainPath + '\mtx.npy')
dist = numpy.load(mainPath + '\dist.npy')

#---------------------------

# CODE
#---------------------------

# print results
print('MTX:')
print(mtx)
print()
print('DIST:')
print(dist)
print()

# start video capture - webcam
cap = cv2.VideoCapture()
cap.open(0, cv2.CAP_DSHOW)

while(True):
    ret, frame = cap.read()
    #frame = rescale_frame(frame, 150)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, (chessboard[0], chessboard[1]), None)

    if ret == True:
        corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)

        # Find the rotation and translation vectors.
        _, rvecs, tvecs, inliers = cv2.solvePnPRansac(objectPoint, corners2, mtx, dist)

        # project 3D points to image plane
        imgpts, jac = cv2.projectPoints(axis, rvecs, tvecs, mtx, dist)

        frame = draw2(frame, imgpts)
    
    cv2.imshow('FRAME_3D', frame)
    k = cv2.waitKey(1) & 0xFF

    if k == ord('s') or k == ord('S'):
        now = datetime.now()
        currentTime = now.strftime("%H-%M-%S")
        cv2.imwrite(imagesOutputPath + '\camera_3D_' + currentTime + '.png', frame)

    if k == ord('q') or k == ord('Q'):
        break


cap.release()
cv2.destroyAllWindows()

#---------------------------

MTX:
[[951.50264721   0.         645.78565699]
 [  0.         950.60546197 376.17484308]
 [  0.           0.           1.        ]]

DIST:
[[-5.11023201e-02  1.07492444e+00 -3.68309691e-04  2.03247023e-03
  -3.31324985e+00]]

